<a href="https://colab.research.google.com/github/EmilSeyfullayev/Mercedes/blob/main/5_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !cat adc.json

{
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//04J5MjxR0gOA8CgYIARAAGAQSNwF-L9Ir_GKvv9SaXL58BsCsAj3NO-4uTGdDc9QIuE8I0eZNv5eJKg7k7d5f600sgwJv-sToG18",
  "revoke_uri": "https://accounts.google.com/o/oauth2/revoke",
  "scopes": [
    "openid",
    "https://www.googleapis.com/auth/userinfo.email",
    "https://www.googleapis.com/auth/cloud-platform",
    "https://www.googleapis.com/auth/appengine.admin",
    "https://www.googleapis.com/auth/compute",
    "https://www.googleapis.com/auth/accounts.reauth",
    "https://www.googleapis.com/auth/drive"
  ],
  "token_uri": "https://oauth2.googleapis.com/token",
  "type": "authorized_user"
}

In [ ]:
!pip install -U -q PyDrive

import httplib2
import json

from google.colab import auth
from oauth2client import GOOGLE_REVOKE_URI, GOOGLE_TOKEN_URI, client
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth_key = {
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//04J5MjxR0gOA8CgYIARAAGAQSNwF-L9Ir_GKvv9SaXL58BsCsAj3NO-4uTGdDc9QIuE8I0eZNv5eJKg7k7d5f600sgwJv-sToG18"
}

credentials = client.OAuth2Credentials(
    access_token=None,
    client_id=auth_key['client_id'],
    client_secret=auth_key['client_secret'],
    refresh_token=auth_key['refresh_token'],
    token_expiry=None,
    token_uri=GOOGLE_TOKEN_URI,
    user_agent=None,
    revoke_uri=GOOGLE_REVOKE_URI)

credentials.refresh(httplib2.Http())
credentials.authorize(httplib2.Http())
cred = json.loads(credentials.to_json())
cred['type'] = 'authorized_user'

with open('adc.json', 'w') as outfile:
  json.dump(cred, outfile)

auth.authenticate_user()

In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

ws = gc.open("MercedesDataBase").worksheet("sheet_1")
db_data = gd.get_as_dataframe(ws)


data = db_data

##Preprocessing
df = pd.DataFrame()

#1 #2
df['city'] = data['Şəhər']
df['car_name'] = data['Marka']

#3
model_1 = []
model_2 = []
model_3 = []

for i in data['Model']:
  model_1.append(str(i).split(" ")[0])

  try:
    model_2.append(str(i).split(" ")[1])
  except IndexError:
    model_2.append(0)

  try:
    model_3.append(str(i).split(" ")[2])
  except IndexError:
    model_3.append(0)

df['model_1'] = model_1
df['model_2'] = model_2
df['model_3'] = model_3


#4, 5, 6
df['year'] = pd.to_numeric(data['Buraxılış ili'])
df['car_type'] = data['Ban növü']
df['color'] = data['Rəng']


#7
engine_volume = []

for i in data['Mühərrik']:
  engine_volume.append(str(i).split(" L")[0])

df['engine_volume'] = pd.to_numeric(engine_volume)

#8
horse_power = []

for i in data['Mühərrikin gücü']:
  horse_power.append(str(i).split(" a.g.")[0])

df['horse_power'] = pd.to_numeric(horse_power)

#9
df['fuel_type'] = data['Yanacaq növü']

#10
went = []

for i in data['Yürüş']:
  went.append(str(i).split(" km")[0].replace(" ", ""))

df['went'] = pd.to_numeric(went)

#11 #12 #13
df['gearbox'] = data['Sürətlər qutusu']
df['transmission'] = data['Ötürücü']
df['new'] = data['Yeni']

#14 #15
currency = []
price = []

for i in data['Qiymət']:
  if str(i).split(" ")[-1] in ["AZN", "$"]:
    currency.append(str(i).split(" ")[-1])
  elif str(i).split(" ")[-1] not in ["AZN", "$"]:
    currency.append("€")

for i in data['Qiymət']:
  if str(i).split(" ")[-1] in ["AZN", "$"]:
    price.append("".join(str(i).split(" ")[0:-1]))
  elif str(i).split(" ")[-1] not in ["AZN", "$"]:
    price.append(i)

df['currency'] = currency
df['price'] = pd.to_numeric(price)

#16

date = []
time_hour = []
time_minute = []

for i in data['Tarix_Saat']:
  date.append(i.split(" ")[0])
  time_hour.append(str(i).split(" ")[1].split(":")[0])
  time_minute.append(str(i).split(" ")[1].split(":")[1])

df['date'] = pd.to_datetime(date, format="%d.%m.%Y" )
df['time_hour'] = pd.to_numeric(time_hour)
df['time_minute'] = pd.to_numeric(time_minute)

#17

values = ['products-i', 'vipped', 'featured', 'salon']

vip = []
featured = []
salon = []

for value in data['Kateqoriya']:
#########################################
  if values[1] in value:
    vip.append(1)
  elif values[1] not in value:
    vip.append(0)
########################################
for value in data['Kateqoriya']:
  if values[2] in value:
    featured.append(1)
  elif values[2] not in value:
    featured.append(0)
########################################
for value in data['Kateqoriya']:
  if values[3] in value:
    salon.append(1)
  elif values[3] not in value:
    salon.append(0)

df['vip'] = vip
df['featured'] = featured
df['salon'] = salon

#18 #19 #20 #21 #22
df['number_of-looks'] = pd.to_numeric(data['Baxışların sayı'])
df['id'] = pd.to_numeric(data['Məhsulun id-si'])
df['link'] = data['Link']
df['loan'] = data['Kredit mümkündür']
df['barter'] = data['Barter mümkündür']

#23 #24
# this one is tricky
#length of one string element (1 number is 15)
#lrngth of others (2 and more number counts) is roughly equals or less than 4 (4 numbers)

cell_phone_number = []

for i in data['Əlaqə nömrəsi']:
  if len(str(i)) > 14:
    cell_phone_number.append(i)
  else:
    cell_phone_number.append(", ".join(str(i)))

df['cell_phone_number'] = cell_phone_number
df['name_of_auto_salon'] = data['Avtosalon adı']

#25
extra_values = ['Yüngül lehimli disklər',
 'ABS', 'Lyuk', 'Yağış sensoru', 'Mərkəzi qapanma',
 'Park radarı', 'Kondisioner', 'Oturacaqların isidilməsi',
 'Dəri salon', 'Ksenon lampalar', 'Arxa görüntü kamerası',
 'Yan pərdələr', 'Oturacaqların ventilyasiyası']

alloy_wheels = [];    abs_auto_block = [];        luke = []
rain_sensors = [];    central_door_blocking = []; parking_radars = []
air_conditioner = []; seat_heating = [];          leather_interior = []
xenon_lights = [];    back_side_camera = [];      side_curtains = []
seat_ventilation = []

for i in data['Ekstra parametrlər']:
####disks
  if extra_values[0] in str(i):
    alloy_wheels.append(1)
  elif extra_values[0] not in str(i):
    alloy_wheels.append(0)

####abs
  if extra_values[1] in str(i):
    abs_auto_block.append(1)
  elif extra_values[1] not in str(i):
    abs_auto_block.append(0)


####luke
  if extra_values[2] in str(i):
    luke.append(1)
  elif extra_values[2] not in str(i):
    luke.append(0)

#rain_sensors
  if extra_values[3] in str(i):
    rain_sensors.append(1)
  elif extra_values[3] not in str(i):
    rain_sensors.append(0)

#central_door_blocking
  if extra_values[4] in str(i):
    central_door_blocking.append(1)
  elif extra_values[4] not in str(i):
    central_door_blocking.append(0)

#parking_radars
  if extra_values[5] in str(i):
    parking_radars.append(1)
  elif extra_values[5] not in str(i):
    parking_radars.append(0)

#air_conditioner
  if extra_values[6] in str(i):
    air_conditioner.append(1)
  elif extra_values[6] not in str(i):
    air_conditioner.append(0)

#seat_heating
  if extra_values[7] in str(i):
    seat_heating.append(1)
  elif extra_values[7] not in str(i):
    seat_heating.append(0)

#leather_interior
  if extra_values[8] in str(i):
    leather_interior.append(1)
  elif extra_values[8] not in str(i):
    leather_interior.append(0)

#xenon_lights
  if extra_values[9] in str(i):
    xenon_lights.append(1)
  elif extra_values[9] not in str(i):
    xenon_lights.append(0)

#back_side_camera
  if extra_values[10] in str(i):
    back_side_camera.append(1)
  elif extra_values[10] not in str(i):
    back_side_camera.append(0)

#side_curtains
  if extra_values[11] in str(i):
    side_curtains.append(1)
  elif extra_values[11] not in str(i):
    side_curtains.append(0)

#seat_ventilation
  if extra_values[12] in str(i):
    seat_ventilation.append(1)
  elif extra_values[12] not in str(i):
    seat_ventilation.append(0)


df['alloy_wheels'] = alloy_wheels;                        df['abs_auto_block'] = abs_auto_block
df['luke'] = luke;                                        df['rain_sensors'] = rain_sensors
df['central_door_blocking'] = central_door_blocking;      df['parking_radars'] = parking_radars
df['air_conditioner'] = air_conditioner;                  df['seat_heating'] = seat_heating
df['leather_interior'] = leather_interior;                df['xenon_lights'] = xenon_lights
df['back_side_camera'] = back_side_camera;                df['side_curtains'] = side_curtains
df['seat_ventilation'] = seat_ventilation

#26
df['product_text'] = data['Məhsulun mətni']

# if any(s in line for s in ('string1', 'string2', ...)): # this is to check words in text by keywords

#27
geolocation_horizontal = []
geolocation_vertical = []

for i in data['Yerləşmə məkanı']:
  try:
    geolocation_horizontal.append(float(i.split("%2C")[0]))
    geolocation_vertical.append(float(i.split("%2C")[1]))
  except AttributeError:
    geolocation_horizontal.append(i)
    geolocation_vertical.append(i)   

df['geolocation_horizontal'] = geolocation_horizontal
df['geolocation_vertical'] = geolocation_vertical

#28 #29
df['parsing_date'] = data['Parsinq olunan tarix']
df['parsing_exact_time'] = data['Parsinq olunan zaman']

price_manat_2021 = []

for i in range(len(df)):
  if df['currency'][i] == 'AZN':
    price_manat_2021.append(df['price'][i])
  elif df['currency'][i] == '$':
    price_manat_2021.append(df['price'][i]*1.7)
  elif df['currency'][i] == '€':
    price_manat_2021.append(df['price'][i]*2)
  else:
    price_manat_2021.append(0)

df['price_manat_2021'] = price_manat_2021

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")

In [ ]:
sentences = []

for i in df['product_text']:
  sentences.append(i)

In [ ]:
#This is done make numerics as strings
sentences = [str (item) for item in sentences]

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))

48266


In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
padded = pad_sequences(sequences, padding='post')

In [ ]:
sentences[5]

'Masin ela veziyyetdedi.her seyi islek veziyyetdedir.problemi demek olarki yoxdu.rengi qara 040 temiz qaradi zersiz.bir dest de qara orginal disk tekerde verecem teze kimi.iki dest ayaqalti hem rezin hem de ozunun orginal kovriki.kalonkalari da deyisilib teze qoyulub.qeza nisani domkrat ve zapas tekeri de ustundedi.xadavoyunda her sey deyisilib orginal qoyulub kitay yox hamisi alman zapcasti ile deyisilib.akumlatoru da almaniyanindi,benzin nasosu da bocsh firmasi 220 manata aldim hemcinin akumulatoru 240 manata almisam.udari yoxdu.ustundeki nomre satilmir.masin oz adimdadi.benzin de az isledir maksimum yayda seher ici 11 amma trasda 7 lt.amerikanka plonkalar ve perdesi de var her seyin ustunde verirem.'

In [ ]:
word_index['veziyyetdedi']

74

In [ ]:
word_index['her']

63

In [ ]:
padded[5]

array([    5,    23,    74,    63,   637,   422,    18,    11,   634,
         669,     6,   431,   351,  2576,   125,  7258, 20726,     2,
        9219,   289,   351,   171,   349,  6062,  1119,    31,    43,
         290,  9219,  5258,   890,  6063,   890,   289,  7259,   171,
       20727,  9220,   169,   317,    31,   105,  5259, 20728, 20729,
          15,  3237,  1201,   289,  2414,   954,    63,   891,   317,
         171,   105,  8103,   492,   530,   569, 20730,    72,   317,
       11059,   169, 20731,   218,  1658,   169, 14050,  4208,  1160,
        1266,  8104,  5605, 20732,  1822,  1266,  1248,    56,     6,
        4961,   493,  1679,     5,    62,  1304,   218,   289,   205,
        1702,  1097,  3555,   837,   697,   675,   825,   438,   109,
        1249,  1431, 20733,    15,  6619,   289,     9,    63, 14051,
         539,  2028,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
pd.DataFrame(padded)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884
0,66,14,2,14048,164,3,19,23,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,4,23,86,150,82,3,4028,792,384,622,153,112,49,1724,13,20723,48,47,3025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1414,9,20724,14,2,82,6,13,146,231,11058,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,142,143,138,144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3026,1586,1532,1182,1159,4394,3442,20725,14049,3027,2733,2307,555,2947,555,4633,249,1127,651,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27903,11,524,172,48262,124,31,8277,31,734,218,4324,31,734,48263,31,317,218,1036,15,2868,31,317,48264,2,334,172,151,5115,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27904,4,44,12,24,1437,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27905,49,156,121,101,1203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27906,5,119,18,8,2,1351,98,6,70,72,2914,5,48265,2057,25,48266,7,29,1375,13,149,26,223,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from operator import itemgetter

In [ ]:
sorted(tokenizer.word_counts.items(), key=itemgetter(1), reverse = True)[0:100]

[('bir', 12844),
 ('yoxdur', 9768),
 ('maşın', 7352),
 ('masin', 6881),
 ('yoxdu', 6161),
 ('mator', 6000),
 ('hec', 5921),
 ('var', 5737),
 ('ideal', 5718),
 ('problemi', 4884),
 ('vəziyyətdədir', 4688),
 ('real', 4329),
 ('heç', 3929),
 ('ve', 3876),
 ('2', 3862),
 ('və', 3767),
 ('veziyyetdedir', 3747),
 ('tam', 3518),
 ('salam', 3498),
 ('narahat', 3301),
 ('etmir', 3244),
 ('ela', 3131),
 ('barter', 3097),
 ('yeni', 3060),
 ('endirim', 2897),
 ('olar', 2882),
 ('cox', 2657),
 ('karopka', 2531),
 ('olunub', 2391),
 ('teze', 2359),
 ('avtomobil', 2358),
 ('salon', 2318),
 ('satilir', 2295),
 ('arxa', 2261),
 ('xerc', 2208),
 ('4', 2157),
 ('udar', 2143),
 ('mawin', 2140),
 ('teleb', 2105),
 ('əla', 2086),
 ('tecili', 2072),
 ('kimi', 2015),
 ('yaxşı', 1886),
 ('ciddi', 1863),
 ('1', 1850),
 ('etsin', 1696),
 ('zeng', 1689),
 ('çox', 1685),
 ('motor', 1679),
 ('karobka', 1667),
 ('bilersiz', 1662),
 ('olacaq', 1637),
 ('vuruq', 1630),
 ('full', 1595),
 ('udari', 1559),
 ('tələb', 151

In [ ]:
word_index['telebbetmir']

17218

In [ ]:
tokenizer.word_counts['qiymetde']

954

In [ ]:
tokenizer.word_counts['razilasmaq']

313

In [ ]:
tokenizer.word_counts['olar']

2882

In [ ]:
tokenizer.word_counts['maşın']

3244

In [ ]:
tokenizer.word_counts['maşin']

456

In [ ]:
tokenizer.word_counts['mashin']

302

In [ ]:
tokenizer.word_counts['masin']

3041

In [ ]:
tokenizer.word_counts['mawin']

957

In [ ]:
3244+3041+957

7242

In [ ]:
round(3244/7242*100, 1) #masin

44.8

In [ ]:
round(3041/7242*100, 1) #maşın

42.0

In [ ]:
round(957/7242*100, 1) #mawin

13.2

In [ ]:
tokenizer.word_counts['yoxdur']

4453

In [ ]:
tokenizer.word_counts['hec'] + tokenizer.word_counts['heç']

4516

In [ ]:
tokenizer.word_counts['hər']

381